# Files

## app.py
### Description
The app.py script is a Flask application designed for anomaly detection and monitoring in transaction data. It incorporates a machine learning model (Isolation Forest) to predict anomalies and utilizes a SQLite database to store transaction records. The script exposes several endpoints for receiving transactions, retrieving transaction data, and checking alerts based on predefined thresholds.
### Endpoints
#### 1. /
Method: GET

Description: Renders the index page.
#### 2. /receive_transaction
Method: POST

Description: Receives transaction data, predicts anomalies, stores transaction records, and returns a JSON response with anomaly test results.
##### Request Payload
```json
{
    "time": "hh:mm",
    "hour": 12,
    "minute": 30,
    "approved": 100,
    "backend_reversed": 0,
    "denied": 5,
    "failed": 2,
    "processing": 3,
    "refunded": 10,
    "reversed": 1,
    "total": 120,
    "success_rate": 0.85,
    "denial_rate": 0.04,
    "reversal_rate": 0.01,
    "failure_rate": 0.02
}
```
##### Response
```json
{
    "time": "hh:mm",
    "score_test": [
        {
            "is_anomalous": "true",
            "value": -0.123,
            "description": "model score"
        }
    ],
    "reversal_test": [
        {
            "is_anomalous": "false",
            "value": 0.01,
            "description": "reversal rate"
        }
    ],
    ...
}

```
#### 3. /api/transactions
Method: GET

Description: Retrieves all transactions from the database.
##### Response
```json
[
    {
        "id": 1,
        "time": "hh:mm",
        "hour": 12,
        ...
    },
    ...
]
```
### Database
The application uses an SQLite database named transactions.db to store transaction records. The Transaction table schema includes fields such as time, hour, minute, approved, denied, failed, reversed, total, success_rate, denial_rate, reversal_rate, failure_rate, and z_score.
### Email alerts
The application implements an example function send_email_alert to send email alerts when certain triggers are met. Replace this function with your actual email alert implementation using a suitable library like Flask-Mail or smtplib.

## monitoring_model.py
### Overview
This script demonstrates the process of training an Isolation Forest model using transaction data and saving the trained model to a file using Joblib. It involves data preprocessing, model creation, fitting the model with training data, and saving the model for future use.
### Script Workflow
#### Import Dependencies

- pandas: For data manipulation.
- Pivot (custom module): For pivoting data (not included in this script).
- IsolationForest from sklearn.ensemble: To create the Isolation Forest model.
- dump from joblib: For saving the model.
#### Load Data

Reads transaction data from a CSV file hosted on GitHub (transactions_1.csv).
#### Data Preprocessing

Uses the custom pivot_df function from the Pivot module to pivot the DataFrame (X) and create a training DataFrame (X_train).
#### Feature Selection

Selects relevant columns (cols) from the training DataFrame to train the model. These columns include features such as approved, backend_reversed, denied, failed, processing, reversed, total, success_rate, reversal_rate, denial_rate, failure_rate, hour, and minute.
#### Model Creation

Initializes an Isolation Forest model with a random state of 42.
#### Model Training

Fits the Isolation Forest model with the selected features (cols) from the training DataFrame (X_train).
#### Save Model

Saves the trained Isolation Forest model to a file named isolation_forest_model.joblib using the dump function from Joblib.

#### Print Confirmation:

Outputs a message indicating that the model has been saved successfully.

## pivoter.py

### Overview

The `pivoter.py` module contains a class `Pivot` with a static method `pivot_df` that performs data pivoting and calculates additional metrics based on transaction data. It is designed to transform a DataFrame by pivoting it based on specific columns and computing rates such as success rate, denial rate, reversal rate, and failure rate.

### Dependencies

- **pandas**: Required for data manipulation and handling.

### Class: Pivot

#### Method: `pivot_df`

- **Description:** Performs data pivoting and computes additional metrics for transaction data.
- **Parameters:**
  - `df` (DataFrame): The input DataFrame containing transaction data.
- **Returns:**
  - `pivoted` (DataFrame): The pivoted DataFrame with calculated rates and additional columns.

#### Data Pivoting

The `pivot_df` method pivots the input DataFrame (`df`) based on the following columns:

```python
index='time', columns='status', values='f0_', fill_value=0
```

### Calculated Metrics
The pivoted DataFrame (pivoted) includes the following calculated metrics:

- total: Sum of values across all status columns.
- success_rate: Ratio of 'approved' transactions to total transactions.
- denial_rate: Ratio of 'denied' transactions to total transactions.
- reversal_rate: Ratio of 'reversed' and 'backend_reversed' transactions to total transactions.
- failure_rate: Ratio of 'failed' transactions to total transactions.

### Additional Processing
The pivoted DataFrame undergoes additional processing steps:

1. Resetting the index and creating new columns for 'hour' and 'minute' based on the 'time' column.
2. Filling any NaN values in the DataFrame with 0.
### Usage
Import the Pivot class from pivoter module:
```python
from pivoter import Pivot
```
Use the pivot_df method to pivot and process transaction data:
```python
pivoted_df = Pivot.pivot_df(input_df)
```
Use the processed DataFrame (pivoted_df) for further analysis or modeling.

### Example
```python
import pandas as pd
from pivoter import Pivot

# Load data into DataFrame (df)

# Pivot and process the DataFrame
pivoted_df = Pivot.pivot_df(df)

# Use the processed DataFrame for further analysis or modeling

```


## simulator.py
### Overview

The `simulator.py` script simulates a scenario where POS (Point of Sale) data is read from a CSV file, pivoted using a custom library/module (`pivoter`), and sent to an API endpoint for processing. It uses pandas for data handling, requests for making API calls, and time for adding delays between API requests.

### Dependencies

- pandas: For data manipulation and handling.
- requests: For making HTTP requests to the API.
- pivoter: A custom library/module (assumed to exist) for data pivoting.

### Script Workflow
1. Import Dependencies:

- pandas: For data manipulation.
- requests: For making HTTP requests.
- Pivot (custom module): For pivoting data (assumed to exist).

2. Read POS Data:

Reads POS transaction data from a CSV file (transactions_2.csv) hosted online.

3. API Configuration:

Defines the API URL (url) where the transaction data will be sent (http://127.0.0.1:5000/receive_transaction).

4. Data Pivoting:

Uses the pivot_df method from the Pivot module to pivot the POS transaction data.

5. API Requests:

- Iterates over each row in the pivoted DataFrame (X_pivoted).
- Converts each row to a dictionary (row_data).
- Sends an HTTP POST request to the API endpoint (url) with the JSON data (row_data).

6. Handling API Responses:

- Checks if the API call was successful (status code 200).
    - If successful, prints the response JSON.
    - If unsuccessful, prints the status code and response text.
- Adds a delay of 10 seconds between API requests using time.sleep(10).

### Usage
- Ensure that the CSV file (transactions_2.csv) containing POS transaction data is accessible and has the required format.
- Make sure the API endpoint (http://127.0.0.1:5000/receive_transaction) is running and ready to receive data.
- Run the script (simulator.py) using Python to simulate sending POS transaction data to the API.
### Additional Notes
Replace pivoter with the actual name of the custom library/module for data pivoting.
Adjust the API URL (url) to point to the correct endpoint where you want to send the transaction data.